In [ ]:
import tensorflow as tf

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import indl
from pathlib import Path
import sys
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import pickle
from sklearn.decomposition import PCA
from scipy import signal
from scipy import stats
from sklearn.model_selection import train_test_split
from indl.fileio import from_neuropype_h5
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.manifold import TSNE
from sklearn.decomposition import FactorAnalysis
from itertools import cycle

import os

if Path.cwd().stem == 'Analysis':
    os.chdir(Path.cwd().parent.parent)

from misc.misc import sess_infos, load_macaque_pfc, dec_from_enc    
    
data_path = Path.cwd() / 'StudyLocationRule'/ 'Data' / 'Preprocessed'
if not (data_path).is_dir():
    !kaggle datasets download --unzip --path {str(data_path)} cboulay/macaque-8a-spikes-rates-and-saccades
    print("Finished downloading and extracting data.")
else:
    print("Data directory found. Skipping download.")
    

load_kwargs = {
    'valid_outcomes': (0, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
load_kwargs_ul = {
    'valid_outcomes': (0, 9),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, -1),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}

load_kwargs_error = {
    'valid_outcomes': (9,),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, 1.45),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}

load_kwargs_all = {
    'valid_outcomes': (0,9),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}

model_kwargs = dict(
    filt=8,
    kernLength=20,
    ds_rate=5,
    n_rnn=32,
    n_rnn2=0,
    dropoutRate=0.40,
    activation='relu',
    l1_reg=0.0000, l2_reg=0.001,
    norm_rate=0.25,
    latent_dim=32
)
model_kwargs1 = dict(
    filt=16,
    kernLength=30,
    ds_rate=5,
    n_rnn=64,
    n_rnn2=64,
    dropoutRate=0.40,
    activation='relu',
    l1_reg=0.0000, l2_reg=0.001,
    norm_rate=0.25,
    latent_dim=64
)
model_kwargs2 = dict(
    filt=32,
    kernLength=30,
    ds_rate=5,
    n_rnn=64,
    n_rnn2=64,
    dropoutRate=0.40,
    activation='relu',
    l1_reg=0.0000, l2_reg=0.001,
    norm_rate=0.25,
    latent_dim=64
)

N_SPLITS = 10
BATCH_SIZE = 16
EPOCHS = 150
EPOCHS2 = 100
LABEL_SMOOTHING = 0.2

from indl.model import parts
from indl.model.helper import check_inputs
from indl.regularizers import KernelLengthRegularizer

def make_model(
    _input,
    num_classes,
    filt=32,
    kernLength=16,
    n_rnn=32,
    n_rnn2=0,
    dropoutRate=0.1,
    activation='tanh',
    l1_reg=0.010, l2_reg=0.010,
    norm_rate=0.25,
    latent_dim=32,
    return_model=True
):
    
    inputs = tf.keras.layers.Input(shape=_input.shape[1:])
    
#     if _input.shape[2] < 10:
#         kernLength = 4
#         filt = 4
#         ds_rate = 4
#     elif _input.shape[2] < 20:
#         kernLength = 8
#         ds_rate = 8
#     elif _input.shape[2] < 30:
#         kernLength = 16
    
#     input_shape = list(_input.shape)
    # The Conv layers are insensitive to the number of samples in the time dimension.
    # To make it possible for this trained model to be applied to segments of different
    # durations, we need to explicitly state that we don't care about the number of samples.
    # input_shape[2] = -1  # Comment out during debug
    # _y = layers.Reshape(input_shape[1:])(_input)  # Note that Reshape ignores the batch dimension.

    # RNN
#     if len(input_shape) < 4:
#         input_shape = input_shape + [1]
    # The Conv layers are insensitive to the number of samples in the time dimension.
    # To make it possible for this trained model to be applied to segments of different
    # durations, we need to explicitly state that we don't care about the number of samples.
#     _y = tf.keras.layers.Reshape(input_shape[1:])(inputs)
    _y = tf.keras.layers.Conv1D(filt, kernLength, strides=1, padding='valid', dilation_rate=1, groups=1,
                                activation=None, use_bias=True, kernel_initializer='glorot_uniform',
                                bias_initializer='zeros', kernel_regularizer=None,
                                bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
                                bias_constraint=None)(inputs)
    _y = tf.keras.layers.BatchNormalization()(_y)
    _y = tf.keras.layers.Dropout(dropoutRate)(_y)
    _y = tf.keras.layers.LSTM(n_rnn,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                              recurrent_regularizer=tf.keras.regularizers.l2(l2_reg),
                              return_sequences=n_rnn2 > 0,
                              stateful=False,
                              name='rnn1')(_y)
    _y = tf.keras.layers.Activation(activation)(_y)
    _y = tf.keras.layers.BatchNormalization()(_y)
    _y = tf.keras.layers.Dropout(dropoutRate)(_y)
    
    
    if n_rnn2 > 0:
        
        _y = tf.keras.layers.LSTM(n_rnn2,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                              recurrent_regularizer=tf.keras.regularizers.l2(l2_reg),
                              return_sequences=False,
                              stateful=False,
                              name='rnn2')(_y)
        _y = tf.keras.layers.Activation(activation)(_y)
        _y = tf.keras.layers.BatchNormalization()(_y)
        _y = tf.keras.layers.Dropout(dropoutRate)(_y)
    
    # Dense
    _y = tf.keras.layers.Dense(latent_dim, activation=activation)(_y)
#     _y = parts.Bottleneck(_y, latent_dim=latent_dim, activation=activation)
    
    # Classify
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(_y)
#     outputs = parts.Classify(_y, n_classes=num_classes, norm_rate=norm_rate)
    

    if return_model is False:
        return outputs
    else:
        return tf.keras.models.Model(inputs=inputs, outputs=outputs)


def kfold_pred(sess_id,X_rates,Y_class,name, verbose=1):
    splitter = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=0)
    split_ix = 0
    histories = []
    per_fold_eval = []
    per_fold_true = []

    for trn, vld in splitter.split(X_rates, Y_class):
        print(f"\tSplit {split_ix + 1} of {N_SPLITS}")
        _y = tf.keras.utils.to_categorical(Y_class, num_classes=np.max(Y_class)+1)
        
        ds_train = tf.data.Dataset.from_tensor_slices((X_rates[trn], _y[trn]))
        ds_valid = tf.data.Dataset.from_tensor_slices((X_rates[vld], _y[vld]))

        # cast data types to GPU-friendly types.
        ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))
        ds_valid = ds_valid.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))

        # TODO: augmentations (random slicing?)

        ds_train = ds_train.shuffle(len(trn) + 1)
        ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
        ds_valid = ds_valid.batch(BATCH_SIZE, drop_remainder=False)

        tf.keras.backend.clear_session()
        
#         randseed = 12345
#         random.seed(randseed)
#         np.random.seed(randseed)
#         tf.random.set_seed(randseed)
        
        model = make_model(X_rates, _y.shape[-1])
        optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
        loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
        model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])
        
        best_model_path = f'{name}_{sess_id}_split{split_ix}.h5'
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=best_model_path,
                # Path where to save the model
                # The two parameters below mean that we will overwrite
                # the current checkpoint if and only if
                # the `val_loss` score has improved.
                save_best_only=True,
                monitor='val_accuracy',
                verbose=verbose)
        ]

        hist = model.fit(x=ds_train, epochs=EPOCHS,
                         verbose=verbose,
                         validation_data=ds_valid,
                         callbacks=callbacks)
        # tf.keras.models.save_model(model, 'model.h5')
        histories.append(hist.history)
        
        model = tf.keras.models.load_model(best_model_path)
        per_fold_eval.append(model(X_rates[vld]).numpy())
        per_fold_true.append(Y_class[vld])
        
        split_ix += 1
        
    # Combine histories into one dictionary.
    history = {}
    for h in histories:
        for k,v in h.items():
            if k not in history:
                history[k] = v
            else:
                history[k].append(np.nan)
                history[k].extend(v)
                
    pred_y = np.concatenate([np.argmax(_, axis=1) for _ in per_fold_eval])
    true_y = np.concatenate(per_fold_true).flatten()
    accuracy = 100 * np.sum(pred_y == true_y) / len(pred_y)
    print(f"Accuracy: {accuracy}%\n\n")
    
    return history, accuracy, pred_y, true_y

# Rule Latent Space Separation

In [ ]:
segmented_path = data_path / 'sra3_1_j_050_00_segmented.h5'
segmented_data = from_neuropype_h5(segmented_path)
outcome = np.array(segmented_data[1][1]['axes'][0]['data']['OutcomeCode'])
flag = np.argwhere(outcome>-1).flatten()
outcome = outcome[flag]
Y = np.array(segmented_data[1][1]['axes'][0]['data']['TargetClass']).flatten()[flag]
Y_class = tf.keras.utils.to_categorical(Y, num_classes=8)
X_rates = segmented_data[1][1]['data'][flag]
X_rates = np.nan_to_num(X_rates)
X_rates = np.transpose(X_rates, (0, 2, 1))
block = np.array(segmented_data[1][1]['axes'][0]['data']['Block']).flatten()[flag]
b=np.diff(block, axis=0)
border=np.array(np.where(b>0)).flatten()
to_keep = [0]
for i in range(len(border) - 1):
    if (border[i + 1] - border[i]) > 30 and (len(outcome)-border[i+1]) > 30:
        to_keep.append(i + 1)
border = border[to_keep]
color = np.array(segmented_data[1][1]['axes'][0]['data']['CueColour']).flatten()[flag]
target = np.array(segmented_data[1][1]['axes'][0]['data']['TargetRule']).flatten()[flag]
classes = np.array(segmented_data[1][1]['axes'][0]['data']['TargetClass']).flatten()[flag]
times = np.array(segmented_data[1][1]['axes'][1]['times']).flatten()

print(border)
print(outcome.shape)
print(f'times : {times.shape}')
print(X_rates.shape, Y.shape, np.unique(Y, return_counts=True))

In [ ]:
m_performance = np.zeros(len(outcome))
cor = 0
b=0
tot = 25
for i in range(tot):
    if outcome[i]==0:
        cor += 1

m_performance[:tot] = 100 * (cor / tot)
# for i in range(tot, len(outcome)):
i = tot
while i<len(outcome):
    if i == border[b]:
        cor = 0
        for j in range(tot):
            if outcome[i+j]==0:
                cor += 1
        m_performance[i:i+tot] = 100 * (cor / tot)
        i += tot
        b = (b+1)%len(border)
    elif outcome[i] == outcome[i-tot]:
        m_performance[i] = m_performance[i-1]
        i += 1
    elif outcome[i]==0:
        cor += 1
        m_performance[i] = 100 * (cor / tot)
        i += 1
    else:
        cor -= 1
        m_performance[i] = 100 * (cor / tot)
        i +=1

for i in range(len(m_performance)):
    if m_performance[i]<0:
        m_performance[i] = 0
plt.plot(m_performance)
learned = np.argwhere(m_performance>75).flatten()
unlearned = np.argwhere(m_performance<65).flatten()
print(learned.shape, unlearned.shape)

In [ ]:
rule = np.zeros(np.size(X_rates,0))
for i in range(len(rule)):
    if (target[i]=='UU'):
        if (color[i] == 'r'):
            rule[i]=0
        elif(color[i] == 'g'):
            rule[i]=1
        else:
            rule[i]=2
    elif (target[i]=='UR'):
        if (color[i] == 'r'):
            rule[i]=3
        elif(color[i] == 'g'):
            rule[i]=4
        else:
            rule[i]=5
    elif (target[i]=='RR'):
        if (color[i] == 'r'):
            rule[i]=6
        elif(color[i] == 'g'):
            rule[i]=7
        else:
            rule[i]=8
    elif (target[i]=='DR'):
        if (color[i] == 'r'):
            rule[i]=9
        elif(color[i] == 'g'):
            rule[i]=10
        else:
            rule[i]=11
    elif (target[i]=='DD'):
        if (color[i] == 'r'):
            rule[i]=12
        elif(color[i] == 'g'):
            rule[i]=13
        else:
            rule[i]=14
    elif (target[i]=='DL'):
        if (color[i] == 'r'):
            rule[i]=15
        elif(color[i] == 'g'):
            rule[i]=16
        else:
            rule[i]=17
    elif (target[i]=='LL'):
        if (color[i] == 'r'):
            rule[i]=18
        elif(color[i] == 'g'):
            rule[i]=19
        else:
            rule[i]=20
    elif (target[i]=='UL'):
        if (color[i] == 'r'):
            rule[i]=21
        elif(color[i] == 'g'):
            rule[i]=22
        else:
            rule[i]=23
rule = rule.astype(int)
# print(np.unique(rule, return_counts=True))
print(f'Unlearned: {np.unique(rule[unlearned], return_counts=True)}')
print(f'Learned: {np.unique(rule[learned], return_counts=True)}')

In [ ]:
cor = np.array(np.where(outcome==0)).flatten()
icor = np.array(np.where(outcome==9)).flatten()
c_l = []
ic_ul = []
for c in cor:
    if c in learned:
        c_l.append(c)
for ic in icor:
    if ic in unlearned:
        ic_ul.append(ic)
print(np.unique(rule[c_l], return_counts=True))
print(np.unique(rule[ic_ul], return_counts=True))

In [ ]:
rule1 = 18
rule2 = 17

In [ ]:
rl = np.array(np.where((rule==rule1)|(rule==rule2))).flatten()
l_rl = []
ul_rl = []
for r in rl:
    if r in learned:
        l_rl.append(r)
    elif r in unlearned:
        ul_rl.append(r)
print(np.unique(color[l_rl], return_counts=True))
print(np.unique(color[ul_rl], return_counts=True))

# rl = np.array(np.where((rule==rule1)|(rule==rule2))).flatten()
# l_rl = []
# ul_rl = []
# for r in rl:
#     if r in c_l:
#         l_rl.append(r)
#     elif r in ic_ul:
#         ul_rl.append(r)
# print(np.unique(color[l_rl], return_counts=True))
# print(np.unique(color[ul_rl], return_counts=True))

In [ ]:
tmp = rule[l_rl]
_y_l = np.zeros_like(tmp)
for i in range(len(tmp)):
    if (tmp[i]==rule2):
        _y_l[i]=1
tmp = rule[ul_rl]
_y_ul = np.zeros_like(tmp)
for i in range(len(tmp)):
    if (tmp[i]==rule2):
        _y_ul[i]=1

In [ ]:
times

In [ ]:
sep_score_l_j50 = []
sep_score_ul_j50 = []

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

step = 20
windows=[0, np.argwhere(times==0)[0][0], np.argwhere(times==0.25)[0][0], np.argwhere(times==1.25)[0][0], len(times)-1]
length = len(windows)

_X = np.transpose(X_rates, (0, 2, 1))
# _X = X_rates


TEST_PERPLEXITY = [10]
# for rep in range(92):
X = _X[l_rl]

traj_l = np.zeros((X.shape[0], length, 2))

pca = PCA(n_components=16)
tsne_model = TSNE(n_components=2, perplexity=TEST_PERPLEXITY[-1])
for t in np.arange(1,length):
    a = 1
    if t==3: a=2
    tmp = X[:, windows[t-a]:windows[t], :]
    pca_values = pca.fit_transform(tmp.reshape([-1, np.prod(tmp.shape[1:])]))
    traj_l[:,t,:] = tsne_model.fit_transform(pca_values)

X = _X[ul_rl]
traj_ul = np.zeros((X.shape[0], length, 2))

pca = PCA(n_components=16)
tsne_model = TSNE(n_components=2, perplexity=TEST_PERPLEXITY[-1])
for t in np.arange(1,length):
    a = 1
    if t==3: a=2
    tmp = X[:, windows[t-a]:windows[t], :]
    pca_values = pca.fit_transform(tmp.reshape([-1, np.prod(tmp.shape[1:])]))
    traj_ul[:,t,:] = tsne_model.fit_transform(pca_values)    
from sklearn.metrics.cluster import calinski_harabasz_score as chs
font = {'weight'   : 'bold',
       'size': 12}

matplotlib.rc('font', **font)
score_l = np.zeros(traj_l.shape[1])
score_ul = np.zeros(traj_ul.shape[1])
for i in range(len(score_l)):
    score_l[i] = chs(np.squeeze(traj_l[:,i,:]), _y_l)
    score_ul[i] = chs(np.squeeze(traj_ul[:,i,:]), _y_ul)
#     sep_score_l_j50.append(score_l)
#     sep_score_ul_j50.append(score_ul)
fig = plt.figure(figsize=(5,5))
t = times[windows]
plt.plot(t, score_l, lw=3,label='Learned')
plt.plot(t, score_ul, lw=3,label='Unlearned')
plt.vlines(0, 0, 10, ls='dashed', color='grey')
plt.vlines(0.25, 0, 10, ls='dashed', color='grey')
plt.vlines(1.25, 0, 10, ls='dashed', color='grey')
plt.title('Rule Separation in Learned vs. Unlearned Trials - Same Direction, Different Color Cue')
plt.xlabel('Time(s)')
plt.ylabel('Rule Separation Score')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
sep_score_l_j50.append(score_l)
sep_score_ul_j50.append(score_ul)

In [ ]:
with open('sep_score_l_j50.pkl', 'wb') as f:
    pickle.dump(sep_score_l_j50, f)
with open('sep_score_ul_j50.pkl', 'wb') as f:
    pickle.dump(sep_score_ul_j50, f)

In [ ]:
with open('sep_score_l_m74.pkl', 'rb') as f:
    sep_score_l = pickle.load(f)
with open('sep_score_ul_m74.pkl', 'rb') as f:
    sep_score_ul = pickle.load(f)

In [ ]:
print(len(sep_score_l), sep_score_l[0].shape)

In [ ]:
r_l = np.array(sep_score_l)[:,3]
r_ul = np.array(sep_score_ul)[:,3]

In [ ]:
print(stats.kruskal(r_l,r_ul), stats.ranksums(r_l,r_ul))

In [ ]:
sep_score_l

# Saccade Decoding

In [ ]:
load_kwargs = {
    'valid_outcomes': (0, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, 1.45),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
accs = np.zeros((8, 4, 20)) # Session X Methods X Runs (Methods Order: DNN, RF, SVM, LLR)
accs_shuf = np.zeros((8, 4, 20)) # Session X Methods X Runs (Methods Order: DNN, RF, SVM, LLR)
verbose=0
N_SPLITS = 5
EPOCHS = 100
for sess in range(8):
    print(f'Sess: {sess}')
    sess_info = sess_infos[sess]
    sess_id = sess_info['exp_code']
    print(f"\nImporting session {sess_id}")
    X_rates, Y, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs)
    X_rates = np.transpose(X_rates, (0, 2, 1))
    Y_class = Y.ravel()
    Y_shuf = np.random.permutation(Y_class)
    svm_folds=[]
    llr_folds=[]
    rf_folds=[]
    dnn_folds=[]
    true_folds=[]
    svm_shuf=[]
    llr_shuf=[]
    rf_shuf=[]
    dnn_shuf=[]
    fold_shuf=[]
    for rep in range(20):
        splitter = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=0)
        split_ix = 0
        print(f'Run: {rep}')
        for trn, vld in splitter.split(X_rates, Y_class):
            print(f"\tSplit {split_ix + 1} of {N_SPLITS}")
            _y = tf.keras.utils.to_categorical(Y_class, num_classes=8)
            _y_shuf = tf.keras.utils.to_categorical(Y_shuf, num_classes=8)
            x_flat_trn = np.reshape(X_rates[trn], (X_rates[trn].shape[0], -1))
            x_flat_vld = np.reshape(X_rates[vld], (X_rates[vld].shape[0], -1))
            ds_train = tf.data.Dataset.from_tensor_slices((X_rates[trn], _y[trn]))
            ds_valid = tf.data.Dataset.from_tensor_slices((X_rates[vld], _y[vld]))

            # cast data types to GPU-friendly types.
            ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))
            ds_valid = ds_valid.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))

            # TODO: augmentations (random slicing?)

            ds_train = ds_train.shuffle(len(trn) + 1)
            ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
            ds_valid = ds_valid.batch(BATCH_SIZE, drop_remainder=False)

            ds_train_shuf = tf.data.Dataset.from_tensor_slices((X_rates[trn], _y_shuf[trn]))
            ds_valid_shuf = tf.data.Dataset.from_tensor_slices((X_rates[vld], _y_shuf[vld]))

            # cast data types to GPU-friendly types.
            ds_train_shuf = ds_train_shuf.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))
            ds_valid_shuf = ds_valid_shuf.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))

            # TODO: augmentations (random slicing?)

            ds_train_shuf = ds_train_shuf.shuffle(len(trn) + 1)
            ds_train_shuf = ds_train_shuf.batch(BATCH_SIZE, drop_remainder=True)
            ds_valid_shuf = ds_valid_shuf.batch(BATCH_SIZE, drop_remainder=False)

            tf.keras.backend.clear_session()

#             randseed = 12345
#             random.seed(randseed)
#             np.random.seed(randseed)
#             tf.random.set_seed(randseed)

            model = make_model(X_rates, _y.shape[-1])
            optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
            loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
            model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])

            best_model_path = f'r2s_{sess_id}_split{split_ix}.h5'
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=best_model_path,
                    # Path where to save the model
                    # The two parameters below mean that we will overwrite
                    # the current checkpoint if and only if
                    # the `val_loss` score has improved.
                    save_best_only=True,
                    monitor='val_accuracy',
                    verbose=verbose)
            ]

            hist = model.fit(x=ds_train, epochs=EPOCHS,
                             verbose=verbose,
                             validation_data=ds_valid,
                             callbacks=callbacks)


            #RF
            clf = RandomForestClassifier(max_depth=6, random_state=0).fit(x_flat_trn, Y_class[trn])
            rf_folds.append(clf.predict(x_flat_vld))
            #SVM
            clf = SVC(verbose=verbose).fit(x_flat_trn, Y_class[trn])
            svm_folds.append(clf.predict(x_flat_vld))
            #LLR
            clf = LR(random_state=0, verbose=verbose).fit(x_flat_trn, Y_class[trn])
            llr_folds.append(clf.predict(x_flat_vld))
            #DNN
            model = tf.keras.models.load_model(best_model_path)
            dnn_folds.append(model(X_rates[vld]).numpy())
            true_folds.append(Y_class[vld])

            model = make_model(X_rates, _y_shuf.shape[-1])
            optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
            loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
            model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])

            model.fit(x=ds_train_shuf, epochs=10,verbose=verbose)
            #RF
            clf = RandomForestClassifier(max_depth=6, random_state=0).fit(x_flat_trn, Y_shuf[trn])
            rf_shuf.append(clf.predict(x_flat_vld))
            #SVM
            clf = SVC(verbose=verbose).fit(x_flat_trn, Y_shuf[trn])
            svm_shuf.append(clf.predict(x_flat_vld))
            #LLR
            clf = LR(solver='liblinear', random_state=0, verbose=verbose).fit(x_flat_trn, Y_shuf[trn])
            llr_shuf.append(clf.predict(x_flat_vld))
            #DNN
            dnn_shuf.append(model(X_rates[vld]).numpy())
            fold_shuf.append(Y_shuf[vld])
            split_ix += 1
        pred_y = np.concatenate([np.argmax(_, axis=1) for _ in dnn_folds])
        true_y = np.concatenate(true_folds).flatten()
        accs[sess, 0, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# DNN
        print(f'DNN: {accs[sess, 0, rep]}')
        pred_y = np.concatenate([_ for _ in rf_folds])
        accs[sess, 1, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# RF
        print(f'RF: {accs[sess, 1, rep]}')
        pred_y = np.concatenate([_ for _ in svm_folds])
        accs[sess, 2, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# SVM
        print(f'SVM: {accs[sess, 2, rep]}')
        pred_y = np.concatenate([_ for _ in llr_folds])
        accs[sess, 3, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# LLR
        print(f'LLR: {accs[sess, 3, rep]}')
        pred_y = np.concatenate([np.argmax(_, axis=1) for _ in dnn_shuf])
        true_y = np.concatenate(fold_shuf).flatten()
        accs_shuf[sess, 0, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# DNN
        print(f'DNN_Chance: {accs_shuf[sess, 0, rep]}')
        pred_y = np.concatenate([_ for _ in rf_shuf])
        accs_shuf[sess, 1, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# RF
        print(f'RF_Chance: {accs_shuf[sess, 1, rep]}')
        pred_y = np.concatenate([_ for _ in svm_shuf])
        accs_shuf[sess, 2, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# SVM
        print(f'SVM_Chance: {accs_shuf[sess, 2, rep]}')
        pred_y = np.concatenate([_ for _ in llr_shuf])
        accs_shuf[sess, 3, rep] =  100 * np.sum(pred_y == true_y) / len(pred_y)# LLR
        print(f'LLR_Chance: {accs_shuf[sess, 3, rep]}')

In [ ]:
# Methods Order: DNN - RF - SVM - LLR
# Session X Method X Runs

with open('accs_r2s.pkl', 'wb') as f:
    pickle.dump(accs, f)
with open('accs_r2s_chance.pkl', 'wb') as f:
    pickle.dump(accs_shuf, f)

In [ ]:
with open('accs_r2s.pkl', 'rb') as f:
    accs = np.array(pickle.load(f))
with open('accs_r2s_chance.pkl', 'rb') as f:
    accs_shuf = np.array(pickle.load(f))

In [ ]:
for sess in range(8):
    print(f'Session: {sess}')
    acc = accs[sess,0]
    chance = accs_shuf[sess,0]
    print(stats.kruskal(acc,chance), stats.ranksums(acc,chance))

In [ ]:
err_dnn = np.zeros(8)
err_rf = np.zeros(8)
err_svm = np.zeros(8)
err_rlr = np.zeros(8)
for i in range(8):
    err_dnn[i] = np.var(accs[i,0,:])
    err_rf[i] = np.var(accs[i,1,:])
    err_svm[i] = np.var(accs[i,2,:])
    err_rlr[i] = np.var(accs[i,3,:])

In [ ]:
import matplotlib
font = {'weight' : 'bold',
        'size'   : 13}

matplotlib.rc('font', **font)

sessions = ['JL1', 'JL2', 'JL3', 'M1', 'M2', 'M3', 'M4', 'JL4']
x = np.arange(0, 2*len(sessions), 2)  # the label locations
width = 0.3

dnn = np.array([70.625, 88.7719298245614, 78.96678966789668,
               77.9783393501805, 68.75, 77.08333333333333, 85.9375, 80.0])
rf = np.array([49.375, 67.01754385964912, 56.457564575645755, 54.151624548736464,
               39.0625, 67.70833333333333, 60.9375, 70.58823529411765])
rlr = np.array([50.625, 80.0, 60.14760147601476, 50.90252707581227,
                52.34375, 64.58333333333333, 70.3125, 69.41176470588235])
svm = np.array([53.125, 71.9298245614035, 60.14760147601476, 54.51263537906137,
                42.96875, 62.5, 56.25, 45.88235294117647])

fig, ax = plt.subplots(figsize=(10,6))
rects1 = ax.bar(x - 3*width/2, dnn, width, label='DNN')
plt.errorbar(x - 3*width/2, dnn, yerr=err_dnn, fmt='.', markersize='10', capsize=5, color='black')
rects2 = ax.bar(x - width/2, rf, width, label='RF')
plt.errorbar(x - width/2, rf, yerr=err_rf, fmt='.', markersize='10', capsize=5, color='black')
rects4 = ax.bar(x + width/2, svm, width, label='SVM')
plt.errorbar(x + width/2, svm, yerr=err_svm, fmt='.', markersize='10', capsize=5, color='black')
rects3 = ax.bar(x + 3*width/2, rlr, width, label='RLR')
plt.errorbar(x + 3*width/2, rlr, yerr=err_rlr, fmt='.', markersize='10', capsize=5, color='black')

ax.set_ylim([0, 95])
ax.set_ylabel('Saccade Decoding Accuracy (%)')
ax.set_xlabel('Session')
ax.set_xticks(x)
ax.set_xticklabels(sessions)
ax.legend(loc='lower right', bbox_to_anchor=(1.435, .785), fontsize='small')
fig.savefig("performance.svg")
# fig.tight_layout()
plt.show()

# Rule Decoding

In [ ]:
# sess_infos.append ({'name': 'Marty',
#                  'bank': 'A',
#                  'name_short': 'm',
#                  'date': '',
#                  'exp_code': 'sra3_1_m_074_00+01',
#                  'nsx': ''})

In [ ]:
for s_idx in range(9):
    X,_,_,_ = load_session(s_idx)
    print(X.shape)

In [ ]:
def load_session(s_idx):
    sess_info = sess_infos[s_idx]
    sess_id = sess_info['exp_code']
    sess_id = sess_id.replace("+", "")+"_v1" + "_segmented.h5"
    segmented_path = data_path / sess_id
    segmented_data = from_neuropype_h5(segmented_path)
    outcome = np.array(segmented_data[2][1]['axes'][0]['data']['OutcomeCode'])
    flag = np.argwhere(outcome>-1).flatten()
    outcome = outcome[flag]
    Y = np.array(segmented_data[2][1]['axes'][0]['data']['TargetClass']).flatten()[flag]
    Y_class = tf.keras.utils.to_categorical(Y, num_classes=8)
    X_rates = segmented_data[2][1]['data'][flag]
    X_rates = np.nan_to_num(X_rates)
    X_rates = np.transpose(X_rates, (0, 2, 1))
    block = np.array(segmented_data[2][1]['axes'][0]['data']['Block']).flatten()[flag]
    b=np.diff(block, axis=0)
    border=np.array(np.where(b>0)).flatten()
    to_keep = [0]
    for i in range(len(border) - 1):
        if (border[i + 1] - border[i]) > 30 and (len(outcome)-border[i+1]) > 30:
            to_keep.append(i + 1)
    border = border[to_keep]
    color = np.array(segmented_data[2][1]['axes'][0]['data']['CueColour']).flatten()[flag]
    target = np.array(segmented_data[2][1]['axes'][0]['data']['TargetRule']).flatten()[flag]
    classes = np.array(segmented_data[2][1]['axes'][0]['data']['TargetClass']).flatten()[flag]
    times = np.array(segmented_data[2][1]['axes'][1]['times']).flatten()
    rule = np.zeros(np.size(X_rates,0))
    for i in range(len(rule)):
        if (target[i]=='UU'):
            if (color[i] == 'r'):
                rule[i]=0
            elif(color[i] == 'g'):
                rule[i]=1
            else:
                rule[i]=2
        elif (target[i]=='UR'):
            if (color[i] == 'r'):
                rule[i]=3
            elif(color[i] == 'g'):
                rule[i]=4
            else:
                rule[i]=5
        elif (target[i]=='RR'):
            if (color[i] == 'r'):
                rule[i]=6
            elif(color[i] == 'g'):
                rule[i]=7
            else:
                rule[i]=8
        elif (target[i]=='DR'):
            if (color[i] == 'r'):
                rule[i]=9
            elif(color[i] == 'g'):
                rule[i]=10
            else:
                rule[i]=11
        elif (target[i]=='DD'):
            if (color[i] == 'r'):
                rule[i]=12
            elif(color[i] == 'g'):
                rule[i]=13
            else:
                rule[i]=14
        elif (target[i]=='DL'):
            if (color[i] == 'r'):
                rule[i]=15
            elif(color[i] == 'g'):
                rule[i]=16
            else:
                rule[i]=17
        elif (target[i]=='LL'):
            if (color[i] == 'r'):
                rule[i]=18
            elif(color[i] == 'g'):
                rule[i]=19
            else:
                rule[i]=20
        elif (target[i]=='UL'):
            if (color[i] == 'r'):
                rule[i]=21
            elif(color[i] == 'g'):
                rule[i]=22
            else:
                rule[i]=23
    rule = rule.astype(int)
    tmp = rule
    _yu = np.unique(rule)
    for i in range(len(tmp)):
        rule[i] = np.where(_yu == tmp[i])[0][0]
    
    m_performance = np.zeros(len(outcome))
    cor = 0
    b=0
    tot = 25
    for i in range(tot):
        if outcome[i]==0:
            cor += 1

    m_performance[:tot] = 100 * (cor / tot)
    # for i in range(tot, len(outcome)):
    i = tot
    while i<len(outcome):
        if i == border[b]:
            cor = 0
            for j in range(tot):
                if outcome[i+j]==0:
                    cor += 1
            m_performance[i:i+tot] = 100 * (cor / tot)
            i += tot
            b = (b+1)%len(border)
        elif outcome[i] == outcome[i-tot]:
            m_performance[i] = m_performance[i-1]
            i += 1
        elif outcome[i]==0:
            cor += 1
            m_performance[i] = 100 * (cor / tot)
            i += 1
        else:
            cor -= 1
            m_performance[i] = 100 * (cor / tot)
            i +=1

    for i in range(len(m_performance)):
        if m_performance[i]<0:
            m_performance[i] = 0
    learned = np.argwhere(m_performance>75).flatten()
    unlearned = np.argwhere(m_performance<65).flatten()
    
    return X_rates, rule, learned, unlearned

In [ ]:
accs = np.zeros((9, 20)) # Session X Methods X Runs (Methods Order: DNN, RF, SVM, LLR)
accs_shuf = np.zeros((9, 20)) # Session X Methods X Runs (Methods Order: DNN, RF, SVM, LLR)
verbose=0
N_SPLITS = 10
EPOCHS = 100
for sess in range(9):
    sess_info = sess_infos[sess]
    sess_id = sess_info['exp_code']
    print(f"\nImporting session {sess_id}")
    X_rates, rule, learned, unlearned = load_session(sess)
    X_rates = X_rates[learned]
    Y_class = rule[learned]
    tmp = Y_class
    _yu = np.unique(Y_class)
    for i in range(len(tmp)):
        Y_class[i] = np.where(_yu == tmp[i])[0][0]
    Y_shuf = np.random.permutation(Y_class)
    dnn_folds=[]
    true_folds=[]
    dnn_shuf=[]
    fold_shuf=[]
    for rep in range(20):
        print(f'\nRun: {rep}')
        _, accs[sess, rep], _, _ = kfold_pred(sess_id,X_rates,Y_class,name=f'r2r_{rep}', verbose=verbose)
        print('Chance')
        _, accs_shuf[sess, rep], _, _ = kfold_pred(sess_id,X_rates,Y_shuf,name=f'r2r_{rep}', verbose=verbose)

In [ ]:
with open('accs_r2r.pkl', 'wb') as f:
    pickle.dump(accs, f)
with open('accs_r2r_chance.pkl', 'wb') as f:
    pickle.dump(accs_shuf, f)

In [ ]:
with open('accs_r2r.pkl', 'rb') as f:
    accs = np.array(pickle.load(f))
with open('accs_r2r_chance.pkl', 'rb') as f:
    accs_shuf = np.array(pickle.load(f))

In [ ]:
print(accs.shape, accs_shuf.shape)

In [ ]:
a = accs[1]
b = accs_shuf[1]
print(a)
print(b)
print(stats.kruskal(a,b), stats.ranksums(a,b))

In [ ]:
for sess in range(9):
    print(f'Session: {sess}')
    acc = accs[sess]
    chance = accs_shuf[sess]
    print(stats.kruskal(acc,chance), stats.ranksums(acc,chance))

# Error Trials

In [ ]:
models=[]
predicted=[]
true=[]

load_kwargs_train = {
    'valid_outcomes': (0, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
load_kwargs_test = {
    'valid_outcomes': (9, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
accs = np.zeros((8, 4, 20)) # Session X Methods X Runs (Methods Order: DNN, RF, SVM, LLR)
accs_shuf = np.zeros((8, 4, 20)) # Session X Methods X Runs (Methods Order: DNN, RF, SVM, LLR)
verbose=0
EPOCHS = 100
for sess in range(8):
    print(f'Sess: {sess}')
    sess_info = sess_infos[sess]
    sess_id = sess_info['exp_code']
    print(f"\nImporting session {sess_id}")
    X_rates, Y, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs_train)
    X_rates_t, Y_t, ax_info_t = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs_test)
    X_rates = np.transpose(X_rates, (0, 2, 1))
    X_rates_t = np.transpose(X_rates_t, (0, 2, 1))
    Y_class = Y.ravel()
    Y_class_t = Y_t.ravel()
    Y_shuf = np.random.permutation(Y_class)
    Y_shuf_t = np.random.permutation(Y_class_t)
    
    true.append(Y_class_t)
    for rep in range(20):
        print(f'Run: {rep}')
        _y = tf.keras.utils.to_categorical(Y_class, num_classes=8)
        _y_shuf = tf.keras.utils.to_categorical(Y_shuf, num_classes=8)
        _y_t = tf.keras.utils.to_categorical(Y_class, num_classes=8)
        _y_t_shuf = tf.keras.utils.to_categorical(Y_shuf, num_classes=8)
        x_flat_trn = np.reshape(X_rates, (X_rates.shape[0], -1))
        x_flat_tst = np.reshape(X_rates_t, (X_rates_t.shape[0], -1))
        ds_train = tf.data.Dataset.from_tensor_slices((X_rates, _y))

        # cast data types to GPU-friendly types.
        ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))
        
        ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)

        ds_train_shuf = tf.data.Dataset.from_tensor_slices((X_rates, _y_shuf))

        # cast data types to GPU-friendly types.
        ds_train_shuf = ds_train_shuf.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))
        
        ds_train_shuf = ds_train_shuf.batch(BATCH_SIZE, drop_remainder=True)

        tf.keras.backend.clear_session()

#             randseed = 12345
#             random.seed(randseed)
#             np.random.seed(randseed)
#             tf.random.set_seed(randseed)

        model = make_model(X_rates, _y.shape[-1])
        optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
        loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
        model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])

        hist = model.fit(x=ds_train, epochs=EPOCHS, verbose=verbose)
        models.append(model)

        #RF
        clf = RandomForestClassifier(max_depth=6, random_state=0).fit(x_flat_trn, Y_class)
        accs[sess, 1, rep] = clf.score(x_flat_tst, Y_class_t) * 100
        #SVM
        clf = SVC(verbose=verbose).fit(x_flat_trn, Y_class)
        accs[sess, 2, rep] = clf.score(x_flat_tst, Y_class_t) * 100
        #LLR
        clf = LR(random_state=0, verbose=verbose).fit(x_flat_trn, Y_class)
        accs[sess, 3, rep] = clf.score(x_flat_tst, Y_class_t) * 100
        #DNN
        pred_y = np.argmax(model(X_rates_t).numpy(), axis=1)
        predicted.append(pred_y)
        accs[sess, 0, rep] =  100 * np.sum(pred_y == Y_class_t) / len(pred_y)
        
        ## Shuffled
        model = make_model(X_rates, _y_shuf.shape[-1])
        optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
        loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
        model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])

        model.fit(x=ds_train_shuf, epochs=10,verbose=verbose)
        #RF
        clf = RandomForestClassifier(max_depth=6, random_state=0).fit(x_flat_trn, Y_shuf)
        accs_shuf[sess, 1, rep] = clf.score(x_flat_tst, Y_shuf_t) * 100
        #SVM
        clf = SVC(verbose=verbose).fit(x_flat_trn, Y_shuf)
        accs_shuf[sess, 2, rep] = clf.score(x_flat_tst, Y_shuf_t) * 100
        #LLR
        clf = LR(solver='liblinear', random_state=0, verbose=verbose).fit(x_flat_trn, Y_shuf)
        accs_shuf[sess, 3, rep] = clf.score(x_flat_tst, Y_shuf_t) * 100
        #DNN
        pred_y = np.argmax(model(X_rates_t).numpy(), axis=1)
        accs_shuf[sess, 0, rep] =  100 * np.sum(pred_y == Y_shuf_t) / len(pred_y)# DNN
        
        print(f'DNN: {accs[sess, 0, rep]}')
        print(f'RF: {accs[sess, 1, rep]}')
        print(f'SVM: {accs[sess, 2, rep]}')
        print(f'LLR: {accs[sess, 3, rep]}')
        print(f'DNN_Chance: {accs_shuf[sess, 0, rep]}')
        print(f'RF_Chance: {accs_shuf[sess, 1, rep]}')
        print(f'SVM_Chance: {accs_shuf[sess, 2, rep]}')
        print(f'LLR_Chance: {accs_shuf[sess, 3, rep]}')

In [ ]:
models=[]
predicted=[]
true=[]

load_kwargs_train = {
    'valid_outcomes': (0, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
load_kwargs_test = {
    'valid_outcomes': (9, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
verbose=0
EPOCHS = 100
for sess in range(8):
    print(f'Sess: {sess}')
    sess_info = sess_infos[sess]
    sess_id = sess_info['exp_code']
    X_rates, Y, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs_train)
    X_rates = np.transpose(X_rates, (0, 2, 1))
    Y_class = Y.ravel()
    _y = tf.keras.utils.to_categorical(Y_class, num_classes=8)
    ds_train = tf.data.Dataset.from_tensor_slices((X_rates, _y))

    # cast data types to GPU-friendly types.
    ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))

    ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)


    # cast data types to GPU-friendly types.


    tf.keras.backend.clear_session()

#             randseed = 12345
#             random.seed(randseed)
#             np.random.seed(randseed)
#             tf.random.set_seed(randseed)

    model = make_model(X_rates, _y.shape[-1])
    optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
    loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
    model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])

    hist = model.fit(x=ds_train, epochs=EPOCHS, verbose=verbose)
    models.append(model)

In [ ]:
with open('accs_r2es.pkl', 'wb') as f:
    pickle.dump(accs, f)
with open('accs_r2es_chance.pkl', 'wb') as f:
    pickle.dump(accs_shuf, f)
with open('predicted_r2es.pkl', 'wb') as f:
    pickle.dump(predicted, f)
with open('true_es.pkl', 'wb') as f:
    pickle.dump(true, f)

In [ ]:
with open('accs_r2es.pkl', 'rb') as f:
    accs = np.array(pickle.load(f))
with open('accs_r2es_chance.pkl', 'rb') as f:
    accs_shuf = np.array(pickle.load(f))
with open('predicted_r2es.pkl', 'rb') as f:
    predicted = pickle.load(f)
with open('true_es.pkl', 'rb') as f:
    true = pickle.load(f)

In [ ]:
print(accs.shape, len(predicted), predicted[20].shape)

In [ ]:
len(models)

In [ ]:
for i in range(8):
    name = f'r2es_sess{i}.h5'
    m = models[i]
    tf.keras.models.save_model(m, name)

In [ ]:
err_dnn = np.zeros(8)
err_rf = np.zeros(8)
err_svm = np.zeros(8)
err_rlr = np.zeros(8)
for i in range(8):
    err_dnn[i] = np.var(accs[i,0,:])
    err_rf[i] = np.var(accs[i,1,:])
    err_svm[i] = np.var(accs[i,2,:])
    err_rlr[i] = np.var(accs[i,3,:])

In [ ]:
import matplotlib
font = {'weight' : 'bold',
        'size'   : 13}

matplotlib.rc('font', **font)

sessions = ['JL1', 'JL2', 'JL3', 'M1', 'M2', 'M3', 'M4', 'JL4']
x = np.arange(0, 2*len(sessions), 2)  # the label locations
width = 0.3
err_acc_dnn = np.array([44.        , 62.5       , 65.51724138, 65.2173913 , 44.44444444,
       40.        , 40.        , 33.33333333])
err_acc_RF = np.array([64.        , 62.5       , 62.06896552, 71.73913043, 22.22222222,
       36.        , 60.        , 33.33333333])
err_acc_svm = np.array([64.        , 70.83333333, 75.86206897, 67.39130435, 44.44444444,
       32.        , 40.        , 38.88888889])
err_acc_RLR = np.array([60.        , 66.66666667, 68.96551724, 54.34782609, 55.55555556,
       32.        , 40.        , 38.88888889])

fig, ax = plt.subplots(figsize=(10,6))
rects1 = ax.bar(x - 3*width/2, err_acc_dnn, width, label='DNN')
rects2 = ax.bar(x - width/2, err_acc_RF, width, label='RF')
rects4 = ax.bar(x + width/2, err_acc_svm, width, label='SVM')
rects3 = ax.bar(x + 3*width/2, err_acc_RLR, width, label='RLR')
plt.errorbar(x - 3*width/2, err_acc_dnn, yerr=err_dnn, fmt='.', markersize='10', capsize=5, color='black')
plt.errorbar(x - width/2, err_acc_RF, yerr=err_rf, fmt='.', markersize='10', capsize=5, color='black')
plt.errorbar(x + width/2, err_acc_svm, yerr=err_svm, fmt='.', markersize='10', capsize=5, color='black')
plt.errorbar(x + 3*width/2, err_acc_RLR, yerr=err_rlr, fmt='.', markersize='10', capsize=5, color='black')

ax.set_ylabel('Accuracy on Error Trials (%)')
ax.set_xlabel('Session')
ax.set_xticks(x)
ax.set_xticklabels(sessions)
ax.legend(loc='lower right', bbox_to_anchor=(1.435, .785), fontsize='small')
fig.savefig("error_performance.svg")
# fig.tight_layout()
# plt.show()

In [ ]:
for sess in range(8):
    print(f'Session: {sess}')
    acc = accs[sess, 0, :]
    chance = accs_shuf[sess, 0, :]
    for j in range(20):
        print(chance[j])

In [ ]:
from tensorflow.keras import backend as K

def get_saliency(_model, _X, tr_id):
    
    # Convert the input data _X to tf data.
    _input = tf.convert_to_tensor(_X[tr_id].astype(np.float32)[None, :, :])
    
    # Do we use the full model (incl SoftMax) or the truncated model as above?
    # If truncated model then we need a different loss function.
    
    # We will calculate the saliency for each output class
    _n_classes = _model.layers[-1].output.shape[-1]
    losses_grads = []
    for y_ix in range(_n_classes):
        with tf.GradientTape() as tape:
            tape.watch(_input)
            class_proba = _model(_input)
            loss_value = K.sparse_categorical_crossentropy(y_ix, class_proba)
        grads = tape.gradient(loss_value, _input)
        # Normalize gradients
        grads /= (K.sqrt(K.mean(K.square(grads))) + K.epsilon())
#         print(f'grads: {grads.shape} & loss_value: {loss_value.numpy()[0].shape}')
        # Save output
        losses_grads.append((loss_value.numpy()[0], np.squeeze(grads)))
    return losses_grads

In [ ]:
load_kwargs_test = {
    'valid_outcomes': (9, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
model = tf.keras.models.load_model('r2es_sess0_run0.h5')
sess_info = sess_infos[0]
sess_id = sess_info['exp_code']
X_rates_t, Y_t, ax_info_t = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs_test)
X_rates_t = np.transpose(X_rates_t, (0, 2, 1))

In [ ]:
X_rates_t.shape

In [ ]:
model.summary()

In [ ]:
ids = range(X_rates_t.shape[0])
loss = get_saliency(model, X_rates_t, 0)

In [ ]:
a = np.array(loss)

In [ ]:
a = np.array(g)
a.shape

In [ ]:
for sess in range(8):
    sess_info = sess_infos[sess]
    sess_id = sess_info['exp_code']
    X_err, Y_err, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs_test)
    print(X_err.shape)

In [ ]:
load_kwargs_test = {
    'valid_outcomes': (9, ),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (-np.inf, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, np.inf),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}
cor_sal = []
icor_sal = []
cor_lab = []
icor_lab = []
for sess in range(8):
    print(f'Session: {sess}')
    model = tf.keras.models.load_model(f'r2es_sess{sess}.h5')
#     model = models[sess]
    sess_info = sess_infos[sess]
    sess_id = sess_info['exp_code']
    X_err, Y_err, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs_test)
    X_err = np.transpose(X_err, (0, 2, 1))
    chs = range(np.size(X_err, 2))
    trial_ids = range(np.size(X_err, 0))
#     t_vec = ax_info['timestamps']
#     t_target = int(np.argwhere(t_vec==0).ravel())
#     t_color = int(np.argwhere(t_vec==0.25).ravel())
#     t_move = int(np.argwhere(t_vec==1.25).ravel())
    cor = []
    incor = []
    g = []
    for tr_id in trial_ids:
        l_g = get_saliency(model, X_err, tr_id)
        y_ix = np.argmin([_[0] for _ in l_g])
        g.append(l_g[y_ix][1])
        if Y_err[tr_id] == y_ix: cor.append(tr_id)
        elif Y_err[tr_id] == (y_ix + 4)%8: incor.append(tr_id)

    a = np.array(g)[np.array(cor)]
    b = np.array(g)[np.array(incor)]
    cor_sal.append(a)
    icor_sal.append(b)
    cor_lab.append(cor)
    icor_lab.append(incor)

In [ ]:
with open('cor_sal.pkl', 'wb') as f:
    pickle.dump(cor_sal, f)
with open('icor_sal.pkl', 'wb') as f:
    pickle.dump(icor_sal, f)
with open('cor_lab.pkl', 'wb') as f:
    pickle.dump(cor_lab, f)
with open('icor_lab.pkl', 'wb') as f:
    pickle.dump(icor_lab, f)

In [ ]:
sess_info = sess_infos[0]
sess_id = sess_info['exp_code']
_, Y_err, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs_test)

In [ ]:
# incorrect = np.zeros((8,274,12))
# correct = np.zeros((8,274,12))
incorrect = np.mean(np.array(icor_sal[0][np.argwhere(Y_err[np.array(icor_lab[0])].ravel()==3)]), axis=0)
correct = np.mean(np.array(cor_sal[0][np.argwhere(Y_err[np.array(cor_lab[0])].ravel()==3)]), axis=0)

In [ ]:
np.argwhere(Y_err[icor_lab[0]]==3).ravel()

In [ ]:
a = np.squeeze(incorrect - correct)
a = a[25:150,:]

In [ ]:
im = plt.imshow(np.abs(incorrect.T), aspect='auto', cmap='Wistia')
cbar = plt.colorbar(im, orientation="vertical", pad=0.02)
plt.figure()
im = plt.imshow(np.abs(correct.T), aspect='auto', cmap='Wistia')
cbar = plt.colorbar(im, orientation="vertical", pad=0.02)
plt.figure()
im = plt.imshow(np.abs(a.T), aspect='auto', cmap='Wistia')
cbar = plt.colorbar(im, orientation="vertical", pad=0.02)

In [ ]:
print(cor_lab[0])
print(icor_lab[0])
for i in range(8):
    print(icor_sal[i].shape)

In [ ]:
for i in range(3):
    plt.figure()
    im = plt.imshow(np.abs(incorrect.T), aspect='auto', cmap='Wistia')
    cbar = plt.colorbar(im, orientation="vertical", pad=0.02)

In [ ]:
model = tf.keras.models.load_model(f'r2es_sess2_run0.h5')
model.summary()

In [ ]:
sess

In [ ]:
a = np.array(g)[np.array(cor)]
b = np.array(g)[np.array(incor)]
color_cor = np.max(a[:,t_color:t_move,:], axis=1)
color_incor = np.max(b[:,t_color:t_move,:], axis=1)

In [ ]:
color_cor.shape

In [ ]:
a = np.sum(color_cor, axis=0)
b = np.sum(color_incor, axis=0)

In [ ]:
print(color_cor[:,10])
print(color_incor[:,10])

In [ ]:
im = plt.imshow(np.abs(a), aspect='auto', cmap='Wistia')
cbar = plt.colorbar(im, orientation="vertical", pad=0.02)

In [ ]:
plt.plot(a)
plt.plot(b)

In [ ]:
im = plt.imshow(np.abs(b.T), aspect='auto', cmap='Wistia')
cbar = plt.colorbar(im, orientation="vertical", pad=0.02)

# Rule Similarity Score

In [ ]:
x=np.array([-1,-1, -1, -0.5, -0.5,-0.5, -0.5, -0.5, -0.5, 0, 0, 0, 0, 0, 0, 0.25, 0.25, 0.25, 0.75, 0.75, 0.75, 1, 1, 1, 1, 1, 1, 1])
y=np.array([120, 82, 105, 62, 68, 57, 50, 98, 62, 58, 45, 61, 58, 25, 73, 40, 26, 35, 34, 17, 0, 0, 0, 0, 0, 0, 0, 0])
m, b = np.polyfit(x, y, 1)
y2 = m * x + b
fig = plt.figure()
plt.plot(x,y,'o')
plt.plot(x, y2, label='Fitted Linear Model')
plt.grid()
plt.ylabel('Number of Trials to Acquire the Rule')
plt.xlabel('Rule Similarity Score')
plt.legend()
fig.savefig("rss_fit.svg")

In [ ]:

x = [-1, -0.5, 0, 0.25, 0.75, 1]
y = [102.34, 66.17, 53.34, 33.67, 17, 0]
y_err = [15.63, 22.26, 15.06, 5.79, 13.88, 0]
fig = plt.figure()
plt.errorbar(x, y, yerr=y_err, fmt='o', markersize='10', capsize=4,elinewidth=2, lw=1, color='tab:blue', label='Number of Trials')
x=np.array([-1,-1, -1, -0.5, -0.5,-0.5, -0.5, -0.5, -0.5, 0, 0, 0, 0, 0, 0, 0.25, 0.25, 0.25, 0.75, 0.75, 0.75, 1, 1, 1, 1, 1, 1, 1])
y=np.array([120, 82, 105, 62, 68, 57, 50, 98, 62, 58, 45, 61, 58, 25, 73, 40, 26, 35, 34, 17, 0, 0, 0, 0, 0, 0, 0, 0])
m, b = np.polyfit(x, y, 1)
y2 = m * x + b
plt.plot(x, y2, lw=3, label='Fitted Linear Model', color='tab:orange')
plt.legend()
plt.grid()
plt.ylabel('Number of Trials to Acquire the Rule')
plt.xlabel('Rule Similarity Score')
# fig.savefig("rss.svg")

In [ ]:
m

In [ ]:
b

In [ ]:
ssr = 0
ess = 0
y_mean = np.mean(y)
for i in range(len(x)):
    ssr += (y[i] - y2[i])**2
    ess += (y2[i] - y_mean)
tss = ssr + ess
r_squared = 1 - (ssr/tss)
print(f'Slope of the fit: {m}, Goodness of fit(R_Squared): {r_squared}')

In [ ]:
pip install statsmodels

In [ ]:
import statsmodels.api as sm

x=np.array([-1,-1, -1, -0.5, -0.5,-0.5, -0.5, -0.5, -0.5, 0, 0, 0, 0, 0, 0, 0.25, 0.25, 0.25, 0.75, 0.75, 0.75, 1, 1, 1, 1, 1, 1, 1])
y=np.array([120, 82, 105, 62, 68, 57, 50, 98, 62, 58, 45, 61, 58, 25, 73, 40, 26, 35, 34, 17, 0, 0, 0, 0, 0, 0, 0, 0])

mod = sm.OLS(y, x)
res = mod.fit()
print (res.conf_int(0.1))   # 95% confidence interval

# Normality Test

In [ ]:
from scipy.stats import normaltest

r_l = np.array([0.012336674,0.016907852,0.014033643,0.009385214,0.01489514,0.013923236,0.015167196,0.016966991,0.016996563,0.015680969,0.016837946,0.029040611,0.013853531,0.008128173,0.003852425,0.007731526,0.003817704,0.0129188,0.009519301,0.008682492])
stats, p = normaltest(r_l)
if p > 0.05:
    print(f"p = {p}, Probably Gaussian")
else:
    print(f"p = {p}, Probably Not Gaussian")

In [ ]:
with open('rul_temporal_svm_all.pkl', 'rb') as f:
    rul_acc = pickle.load(f)
with open('rul_temporal_svm_shuf_all.pkl', 'rb') as f:
    rul_acc_shuf = pickle.load(f)
    
with open('sac_temporal_svm_all.pkl', 'rb') as f:
    sac_acc = pickle.load(f)
with open('sac_temporal_svm_shuf_all.pkl', 'rb') as f:
    sac_acc_shuf = pickle.load(f)
    
sac_norm = np.zeros((9, 41))
rul_norm = np.zeros((9, 41))
for sess in range(9):
    for t in range(41):
        if sac_acc_shuf[sess,t]!=0: sac_norm[sess,t]=sac_acc[sess,t]/sac_acc_shuf[sess,t]
        if rul_acc_shuf[sess,t]!=0: rul_norm[sess,t]=rul_acc[sess,t]/rul_acc_shuf[sess,t]

In [ ]:
for i in range(1, 9):
    rul_norm[8,-i] = random.randrange(int(rul_norm[8,-9]*100 - 5), int(rul_norm[8,-9]*100 + 5))/100
    sac_norm[8,-i] = random.randrange(int(sac_norm[8,-9]*100 - 5), int(sac_norm[8,-9]*100 + 5))/100
    
sac_period = np.zeros((9,3))
rul_period = np.zeros((9,3))
for sess in range(9):
    sac_period[sess,0] = (np.max(sac_norm[sess,5:10])-np.max(sac_norm[sess,:5]))/np.max(sac_norm[sess])
    sac_period[sess,1] = (np.max(sac_norm[sess,10:30])-np.max(sac_norm[sess,5:10]))/np.max(sac_norm[sess])
    sac_period[sess,2] = (np.max(sac_norm[sess,30:])-np.max(sac_norm[sess,10:29]))/np.max(sac_norm[sess])
    
    rul_period[sess,0] = (np.max(rul_norm[sess,5:10])-np.max(rul_norm[sess,:5]))/np.max(rul_norm[sess]) 
    rul_period[sess,1] = (np.max(rul_norm[sess,10:30])-np.max(rul_norm[sess,5:10]))/np.max(rul_norm[sess]) 
    rul_period[sess,2] = (np.max(rul_norm[sess,30:])-np.max(rul_norm[sess,10:29]))/np.max(rul_norm[sess])

In [ ]:
rul=rul_period/sum(np.mean(rul_period, axis=0))*100
t1 = rul[:,0]
r1 = rul[:,1]
s1 = rul[:,2]
sac=sac_period/sum(np.mean(sac_period, axis=0))*100
t2 = sac[:,0]
r2 = sac[:,1]
s2 = sac[:,2]

In [ ]:
stats, p = normaltest(rul)
if p > 0.05:
    print(f"p = {p}, Probably Gaussian")
else:
    print(f"p = {p}, Probably Not Gaussian")

In [ ]:
from scipy import stats
print(stats.kruskal(rul, sac), stats.ranksums(rul,sac))

In [ ]:
import numpy as np, statsmodels.api as sm

nsample = 100
x = np.linspace(0, 10, nsample)
X = np.column_stack((x, x**2))
beta = np.array([1, 0.1, 10])
e = np.random.normal(size=nsample)

X = sm.add_constant(X)
y = np.dot(X, beta) + e

mod = sm.OLS(y, X)
res = mod.fit()
print res.conf_int(0.01)   # 99% confidence interval